In [9]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [10]:
key_path = r"C:\Users\Lenovo M800\Downloads\ecommerce-dataset-462116-cf4b454f1eb3.json"

In [11]:
credentials = service_account.Credentials.from_service_account_file(key_path)

In [34]:
# Initialize BigQuery Client

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [25]:
# Query to fetch data

query = """
SELECT 
  InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country
FROM 
  `ecommerce-dataset-462116.ecommerce.ecommerce data`
"""

In [26]:
#Load data into dataframe

df = client.query(query).to_dataframe()

In [27]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00+00:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00+00:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00+00:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00+00:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00+00:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00+00:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00+00:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00+00:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00+00:00,4.15,12680,France


In [28]:
# Remove missing values(Cleaning)

df.dropna(inplace=True)  
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [29]:
#Feature Engineering

df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')

C:\Users\Lenovo M800\AppData\Local\Temp\ipykernel_7700\3374057179.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')


In [30]:
# Example KPI: Average Order Value

avg_order_value = df.groupby('InvoiceNo')['TotalPrice'].sum().mean()
print("Average Order Value:", avg_order_value)

Average Order Value: 374.0453273546642


In [31]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,YearMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00+00:00,2.55,17850,United Kingdom,15.3,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00+00:00,3.39,17850,United Kingdom,20.34,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00+00:00,2.75,17850,United Kingdom,22.0,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00+00:00,3.39,17850,United Kingdom,20.34,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00+00:00,3.39,17850,United Kingdom,20.34,2010-12
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00+00:00,0.85,12680,France,10.2,2011-12
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00+00:00,2.10,12680,France,12.6,2011-12
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00+00:00,4.15,12680,France,16.6,2011-12
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00+00:00,4.15,12680,France,16.6,2011-12


In [32]:
# Save cleaned CSV for Power BI

df.to_csv('cleaned_ecommerce_data.csv', index=False)